In [ ]:
!pip install pykrige -qqU

In [ ]:
!unzip *.zip

Archive:  Horde_slice 1.dat.zip
  inflating: Horde_slice 1.dat       


In [ ]:
import numpy as np, pandas as pd, cv2 as cv
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from tqdm import trange, tqdm
from PIL import Image

In [ ]:
df = pd.read_csv('Horde_slice 1.dat', sep=' ', header=None, names=['x','y','z'])
df.head()

,x,y,z
0,364128.00000,3632519.0,1.701410e+38
1,364128.05005,3632519.0,1.701410e+38
2,364128.10010,3632519.0,1.701410e+38
3,364128.15015,3632519.0,1.701410e+38
4,364128.20020,3632519.0,1.701410e+38


In [ ]:
min_x,max_x = df['x'].min(),df['x'].max()
min_y,max_y = df['y'].min(),df['y'].max()

grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,0.05), np.arange(min_y,max_y,0.05))
points = df[['x','y']].values
values = df['z'].values
values[values > 1.7e+37] = np.nan

grid = griddata(points, values, (grid_x, grid_y), method='nearest')#, fill_value=np.nan, rescale=False)
pd.DataFrame(zip(grid_x.flatten(), grid_y.flatten(), grid.flatten()), columns=['x','y','z']).to_csv(f'original_grid.csv', index=None)
grid.shape, np.nanmin(grid), np.nanmax(grid), np.unique(grid).shape

((620, 1000), 8885.933, 17982.962, (328901,))

In [ ]:
plt.hist(grid.flatten(),bins=100);

In [ ]:
# _,ax = plt.subplots(1,1,figsize=(20,10))
plt.imshow(grid, origin='lower');

In [ ]:
strides = [1,2,3,5]
wnds = [5, 10, 15, 18, 19, 20, 21, 22, 25]

strides = [5]
wnds = [20]

mx = np.nanmax(grid)

for stride in tqdm(strides):
  for wnd in tqdm(wnds):
    grid_norm, cnts = np.zeros_like(grid),np.zeros_like(grid)

    for row in range(0,grid.shape[0]-wnd,stride):
        for col in range(0,grid.shape[1]-wnd,stride):
            d = grid[row:row+wnd, col:col+wnd]

            grid_norm[row:row+wnd, col:col+wnd] += (d - np.nanmin(d)) / (np.nanmax(d)-np.nanmin(d) + 1e-8)  # 0..1 range
            # grid_norm[row:row+wnd, col:col+wnd] += (d - np.nanmean(d)) / (np.nanstd(d) + 1e-8)            # 0-mean, 1-std
            # grid_norm[row:row+wnd, col:col+wnd] += d / np.sqrt(np.nansum(d**2)+1e-8)                      # L2-norm
            # grid_norm[row:row+wnd, col:col+wnd] += cv.equalizeHist((256*d/mx).astype('uint8')) # histogram equalization

            cnts[row:row+wnd, col:col+wnd] += 1

    grid_norm /= (cnts + 1e-8)

    fname = f'window_{wnd}_stride_{stride}'
    pd.DataFrame(zip(grid_x.flatten(), grid_y.flatten(), grid_norm.flatten()), columns=['x','y','z']).to_csv(f'{fname}.csv', index=None)

    fig,ax = plt.subplots(1,1,figsize=(20,10))
    ax.imshow(grid_norm, origin='lower') #, interpolation='bilinear');
    fig.savefig(f'{fname}.png')

In [ ]:
grid_norm.shape, np.nanmin(grid_norm), np.nanmax(grid_norm), np.unique(grid_norm).shape

In [ ]:
# https://scikit-gstat.readthedocs.io/en/latest/userguide/kriging.html
#from pykrige.uk import UniversalKriging
#from pykrige.ok import OrdinaryKriging

#UK = UniversalKriging(
    #grid_x.flatten(),
   # grid_y.flatten(),
    #grid_norm.flatten(),
   # variogram_model="linear",
   # drift_terms=["regional_linear"],
#)

#OK = OrdinaryKriging(
   # grid_x.flatten(),
    #grid_y.flatten(),
    #grid_norm.flatten(),
    #variogram_model="linear",
   # verbose=False,
    #enable_plotting=False,
#)

# z, ss = UK.execute("grid", np.arange(min_x,max_x,0.04), np.arange(min_y,max_y,0.04))
#z, ss = OK.execute("grid", np.arange(min_x,max_x,0.04), np.arange(min_y,max_y,0.04))


#plt.imshow(z)
#plt.show()

In [ ]:
# grid_norm_24 = (grid_norm*255**3).astype(int)
# out = np.frombuffer(grid_norm_24.tobytes(), dtype='uint8')
# out = out.reshape(*grid_norm_24.shape, -1)[:,:,:3]
# Image.fromarray(out).save('Horde_slice_1.png')

In [ ]:
# fig,ax = plt.subplots(1,1,figsize=(28,14))
# ax.imshow(grid_norm, origin='lower') #, interpolation='bilinear');
# fig.savefig(f'window_{wnd}_stride_{stride}.png')
# # Image.fromarray(grid_norm, mode='L').save('Horde_slice_1.png')

In [ ]:
grid_nan[:,4992:4992+wnd+1]